# pycisTopic analysis

Full dataset, using consensus peak regions.

In [1]:
import pycisTopic
%load_ext nb_black
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import pickle
import pandas as pd
import os

<IPython.core.display.Javascript object>

In [2]:
!pwd

/dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/full_3_cistopic_consensus


<IPython.core.display.Javascript object>

In [3]:
wdir = '/dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/full_3_cistopic_consensus'
os.chdir( wdir )

<IPython.core.display.Javascript object>

In [4]:
# create output directory:
f_final_dir = os.path.join(wdir, 'downstream_analysis')
if not os.path.exists(f_final_dir):
    os.makedirs(f_final_dir)

<IPython.core.display.Javascript object>

In [5]:
import glob

<IPython.core.display.Javascript object>

In [6]:
import re

sample_annot = pd.DataFrame(
    [
        ['Broad_1','BioRad'],
        ['Broad_2','BioRad'],
        ['Stanford_1','10x v1.0'],
        ['Stanford_2','10x v1.0'],
        ['VIB_1','10x v1.0'],
        ['VIB_2','10x v1.0'],
        ['CNAG_1','10x v1.1'],
        ['CNAG_2','10x v1.1'],
        ['Broad_mito_1','mito-scATAC-seq'],
        ['Broad_mito_2','mito-scATAC-seq'],
        ['Sanger_1','10x multiome'],
        ['Sanger_2','10x multiome'],
        ['s3atac','s3-ATAC'],
        ['VIB_Hydrop_1','HyDrop'],
        ['VIB_Hydrop_2','HyDrop'],
        
    ],
    columns=['key','Technology']
)
sample_annot['Sample'] = sample_annot['key']
sample_annot.set_index('key', drop=False, inplace=True)
# rename samples:
sample_annot.loc['Broad_1','Sample'] = 'Harvard_1'
sample_annot.loc['Broad_2','Sample'] = 'Harvard_2'
sample_annot.loc['Broad_mito_1','Sample'] = 'Broad_1'
sample_annot.loc['Broad_mito_2','Sample'] = 'Broad_2'
sample_annot.loc['s3atac','Sample'] = 'OHSU'
sample_annot.loc['VIB_Hydrop_1','Sample'] = 'Hydrop_1'
sample_annot.loc['VIB_Hydrop_2','Sample'] = 'Hydrop_2'

sample_annot['Replicate'] = [ x.Sample.split('_')[-1] for i,x in sample_annot.iterrows() ]
sample_annot['st'] = [ re.sub('_[0-9]','',x.Sample)+' ('+x.Technology+')' for i,x in sample_annot.iterrows() ]

sample_annot.loc[sample_annot['key']=='s3atac','Replicate'] = 1
sample_map = { x: y.Sample for x,y in sample_annot.iterrows() }
sample_annot

,key,Technology,Sample,Replicate,st
key,,,,,
Broad_1,Broad_1,BioRad,Harvard_1,1,Harvard (BioRad)
Broad_2,Broad_2,BioRad,Harvard_2,2,Harvard (BioRad)
Stanford_1,Stanford_1,10x v1.0,Stanford_1,1,Stanford (10x v1.0)
Stanford_2,Stanford_2,10x v1.0,Stanford_2,2,Stanford (10x v1.0)
VIB_1,VIB_1,10x v1.0,VIB_1,1,VIB (10x v1.0)
VIB_2,VIB_2,10x v1.0,VIB_2,2,VIB (10x v1.0)
CNAG_1,CNAG_1,10x v1.1,CNAG_1,1,CNAG (10x v1.1)
CNAG_2,CNAG_2,10x v1.1,CNAG_2,2,CNAG (10x v1.1)
Broad_mito_1,Broad_mito_1,mito-scATAC-seq,Broad_1,1,Broad (mito-scATAC-seq)


<IPython.core.display.Javascript object>

## Save/load cisTopic objects

In [7]:
cto_consensus_paths = sorted(glob.glob(f'cistopic_objects/*consensus.pkl'))
cistopic_obj_path_dict = {x.split('/')[-1].split(f'__')[0]:x for x in cto_consensus_paths}
cistopic_obj_path_dict

{'BIO_ddseq_1.FULL': 'cistopic_objects/BIO_ddseq_1.FULL__cto.scrublet0-4.fmx.singlets.model_20topics.dimreduc.consensus.pkl',
 'BIO_ddseq_2.FULL': 'cistopic_objects/BIO_ddseq_2.FULL__cto.scrublet0-4.fmx.singlets.model_20topics.dimreduc.consensus.pkl',
 'BIO_ddseq_3.FULL': 'cistopic_objects/BIO_ddseq_3.FULL__cto.scrublet0-4.fmx.singlets.model_21topics.dimreduc.consensus.pkl',
 'BIO_ddseq_4.FULL': 'cistopic_objects/BIO_ddseq_4.FULL__cto.scrublet0-4.fmx.singlets.model_19topics.dimreduc.consensus.pkl',
 'BRO_mtscatac_1.FULL': 'cistopic_objects/BRO_mtscatac_1.FULL__cto.scrublet0-4.fmx.singlets.model_18topics.dimreduc.consensus.pkl',
 'BRO_mtscatac_2.FULL': 'cistopic_objects/BRO_mtscatac_2.FULL__cto.scrublet0-4.fmx.singlets.model_16topics.dimreduc.consensus.pkl',
 'CNA_10xmultiome_1.FULL': 'cistopic_objects/CNA_10xmultiome_1.FULL__cto.scrublet0-4.fmx.singlets.model_12topics.dimreduc.consensus.pkl',
 'CNA_10xmultiome_2.FULL': 'cistopic_objects/CNA_10xmultiome_2.FULL__cto.scrublet0-4.fmx.singl

<IPython.core.display.Javascript object>

## Topic binarization & QC

In [8]:
from pycisTopic.topic_binarization import binarize_topics
from pycisTopic.topic_qc import compute_topic_metrics, plot_topic_qc
from pycisTopic.utils import fig2img
import matplotlib.pyplot as plt
from pycisTopic.topic_qc import topic_annotation
from pycisTopic.diff_features import (
    impute_accessibility,
    normalize_scores,
    find_highly_variable_features,
    find_diff_features
)
from pycisTopic.clust_vis import plot_imputed_features
import numpy as np
import copy

<IPython.core.display.Javascript object>

### Binarize the topic-region distributions

In [9]:
for sample in cistopic_obj_path_dict.keys():
# for sample  in ['SAN_10xmultiome_1.FIXEDCELLS']:
    path = cistopic_obj_path_dict[sample]
    if not os.path.exists(f'downstream_analysis/DARs/{sample}'):
        if os.path.isfile(path):
            print(f"Loading {path}")
            with open(path, 'rb') as f:
                cto = pickle.load(f)
                
            cto.cell_names = cto.cell_data.index
            if [x.split('-')[0] for x in cto.selected_model.cell_topic.columns] == [x.split('___')[0] for x in list(cto.cell_names)]:
                cto.selected_model.cell_topic.columns =  list(cto.cell_names)
                
            region_bin_topics = binarize_topics(
                cto,
                method='otsu',
                ntop=3000,
                plot=True,
                num_columns=6,
                save=f'plots_qc/{sample}__topic_region_binarized.png'
            )

            with open(
                    f"downstream_analysis/binarized_topics/{sample}__topic_region_binarized.pkl",'wb'
            ) as f:
                pickle.dump(region_bin_topics, f, protocol=4)

            binarized_cell_topics = binarize_topics(
                cto,
                target='cell',
                method='li',
                plot=True,
                num_columns=5,
                nbins=100,
                save=f'plots_qc/{sample}__cells_topic_binarized.png'
            )

            with open(
                    f"downstream_analysis/binarized_topics/{sample}__cells_topic_binarized.pkl",'wb'
            ) as f:
                pickle.dump(region_bin_topics, f, protocol=4)

            topic_qc_metrics = compute_topic_metrics(cto)

            with open(
                    f"downstream_analysis/binarized_topics/{sample}__topic_qc.pkl",'wb'
            ) as f:
                pickle.dump(topic_qc_metrics, f, protocol=4)

            fig_dict= {
                'CoherenceVSAssignments': plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Log10_Assignments',
                                                        var_color='Gini_index', plot=False, return_fig=True),
                'AssignmentsVSCells_in_bin': plot_topic_qc(topic_qc_metrics, var_x='Log10_Assignments',
                                                           var_y='Cells_in_binarized_topic', var_color='Gini_index',
                                                           plot=False, return_fig=True),
                'CoherenceVSCells_in_bin': plot_topic_qc(topic_qc_metrics, var_x='Coherence',
                                                         var_y='Cells_in_binarized_topic', var_color='Gini_index',
                                                         plot=False, return_fig=True),
                'CoherenceVSRegions_in_bin': plot_topic_qc(topic_qc_metrics, var_x='Coherence',
                                                           var_y='Regions_in_binarized_topic', var_color='Gini_index',
                                                           plot=False, return_fig=True),
                'CoherenceVSMarginal_dist': plot_topic_qc(topic_qc_metrics, var_x='Coherence',
                                                          var_y='Marginal_topic_dist', var_color='Gini_index',
                                                          plot=False, return_fig=True),
                'CoherenceVSGini_index': plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Gini_index',
                                                       var_color='Gini_index', plot=False, return_fig=True)}

            fig=plt.figure(figsize=(40, 43))
            i = 1
            for fig_ in fig_dict.keys():
                plt.subplot(2, 3, i)
                img = fig2img(fig_dict[fig_]) #To convert figures to png to plot together, see .utils.py. This converts the figure to png.
                plt.imshow(img)
                plt.axis('off')
                i += 1
            plt.subplots_adjust(wspace=0, hspace=-0.70)
            #plt.tight_layout()
            plt.show()
            plt.savefig(f'plots_qc/{sample}__topic_qc_metrics.png', facecolor='white', dpi=150)

            topic_annot = topic_annotation(
                cto,
                annot_var='consensus_cell_type',
                binarized_cell_topic=binarized_cell_topics,
                general_topic_thr = 0.2
            )
            topic_qc_metrics = pd.concat(
                [
                    topic_annot[['consensus_cell_type', 'Ratio_cells_in_topic', 'Ratio_group_in_population']],
                    topic_qc_metrics
                ],
                axis=1
            )

            imputed_acc_obj = impute_accessibility(
                cto,
                selected_cells=None,
                selected_regions=None,
                scale_factor=10**6
            )

            normalized_imputed_acc_obj = normalize_scores(
                imputed_acc_obj,
                scale_factor=10**4
            )

            with open(
                f"downstream_analysis/imputed_acc_objs/{sample}__imputed_acc_obs.pkl",'wb'
            ) as f:
                pickle.dump(normalized_imputed_acc_obj, f, protocol=4)

            variable_regions = find_highly_variable_features(
                normalized_imputed_acc_obj,
                min_disp = 0.05,
                min_mean = 0.0125,
                max_mean = 3,
                max_disp = np.inf,
                n_bins=20,
                n_top_features=None,
                plot=True,
                save=f'plots_qc/{sample}__HVR.png'
            )

            print(f"Found {len(variable_regions)} variable regions")
            with open(f'downstream_analysis/HVRs/{sample}__HVRs.bed', 'w') as f:
                for line in [x.replace(':','\t').replace('-','\t') for x in variable_regions]:
                    f.write(f"{line}\n")
                    
            with open(
                f'downstream_analysis/HVRs/{sample}__HVRs.pkl', 'w'
            ) as f:
                pickle.dump(variable_regions, f, protocol=4)

            os.mkdir(f'downstream_analysis/DARs/{sample}')

            markers_dict = find_diff_features(
                cto,
                normalized_imputed_acc_obj,
                variable='consensus_cell_type',
                var_features=variable_regions,
                contrasts=None,
                adjpval_thr=0.05,
                log2fc_thr=np.log2(1.5),
                n_cpu=5,
            )
            with open(
                    f"downstream_analysis/DARs/{sample}/{sample}__DARs_dict.pkl",'wb'
            ) as f:
                pickle.dump(markers_dict, f, protocol=4)

            for cell_type in markers_dict.keys():
                markers = markers_dict[cell_type].index.tolist()
                df = pd.DataFrame(markers)
                if len(df) == 0:
                    print(f"no DARs found for {cell_type} in {sample}")
                else:
                    df[[0,1]] = df[0].str.split(':',expand=True)
                    df[[1,2]] = df[1].str.split('-',expand=True)
                    df[3] = markers_dict[cell_type].index.tolist()
                    df[3] = cell_type.replace(' ', '_')# + '_' + df[3].astype(str)
                    score = markers_dict[cell_type]['Log2FC']
                    score = score.reset_index(drop=True)
                    df[4] = score
                    df[5] = '.'
                    pval = markers_dict[cell_type]['Adjusted_pval']
                    pval = pval.reset_index(drop=True)
                    df[6] = pval
                    name = cell_type.replace(' ', '_')
                    # name = name.replace('/', '-')
                    if not os.path.exists(f'downstream_analysis/DARs/{sample}'):
                        os.mkdir(f'downstream_analysis/DARs/{sample}')

                    df.to_csv(f'downstream_analysis/DARs/{sample}/{sample}__{cell_type.replace(" ", "_")}__DARs.bed', sep='\t', header=False, index=False)
                    df[0:2000].to_csv(f'downstream_analysis/DARs/{sample}/{sample}__{cell_type.replace(" ", "_")}__DARs.TOP2k.bed', sep='\t', header=False, index=False)

        else:
            print(f'{path} does not exist!')
    else:
        print(f"downstream_analysis/DARs/{sample} already exists!")

downstream_analysis/DARs/BIO_ddseq_1.FULL already exists!
downstream_analysis/DARs/BIO_ddseq_2.FULL already exists!
downstream_analysis/DARs/BIO_ddseq_3.FULL already exists!
downstream_analysis/DARs/BIO_ddseq_4.FULL already exists!
downstream_analysis/DARs/BRO_mtscatac_1.FULL already exists!
downstream_analysis/DARs/BRO_mtscatac_2.FULL already exists!
downstream_analysis/DARs/CNA_10xmultiome_1.FULL already exists!
downstream_analysis/DARs/CNA_10xmultiome_2.FULL already exists!
downstream_analysis/DARs/CNA_10xv11_1.FULL already exists!
downstream_analysis/DARs/CNA_10xv11_2.FULL already exists!
downstream_analysis/DARs/CNA_10xv11_3.FULL already exists!
downstream_analysis/DARs/CNA_10xv11_4.FULL already exists!
downstream_analysis/DARs/CNA_10xv11_5.FULL already exists!
downstream_analysis/DARs/CNA_10xv2_1.FULL already exists!
downstream_analysis/DARs/CNA_10xv2_2.FULL already exists!
downstream_analysis/DARs/CNA_hydrop_1.FULL already exists!
downstream_analysis/DARs/CNA_hydrop_2.FULL alrea

<IPython.core.display.Javascript object>

# Calculate DARs between male and female

In [10]:
normalized_imputed_acc_obj_path_dict = {x.split('/')[-1].split(f'__')[0]:x for x in sorted(glob.glob(f'downstream_analysis/imputed_acc_objs/*__imputed_acc_obs.pkl'))}
normalized_imputed_acc_obj_path_dict

{'BIO_ddseq_1.FULL': 'downstream_analysis/imputed_acc_objs/BIO_ddseq_1.FULL__imputed_acc_obs.pkl',
 'BIO_ddseq_2.FULL': 'downstream_analysis/imputed_acc_objs/BIO_ddseq_2.FULL__imputed_acc_obs.pkl',
 'BIO_ddseq_3.FULL': 'downstream_analysis/imputed_acc_objs/BIO_ddseq_3.FULL__imputed_acc_obs.pkl',
 'BIO_ddseq_4.FULL': 'downstream_analysis/imputed_acc_objs/BIO_ddseq_4.FULL__imputed_acc_obs.pkl',
 'BRO_mtscatac_1.FULL': 'downstream_analysis/imputed_acc_objs/BRO_mtscatac_1.FULL__imputed_acc_obs.pkl',
 'BRO_mtscatac_2.FULL': 'downstream_analysis/imputed_acc_objs/BRO_mtscatac_2.FULL__imputed_acc_obs.pkl',
 'CNA_10xmultiome_1.FULL': 'downstream_analysis/imputed_acc_objs/CNA_10xmultiome_1.FULL__imputed_acc_obs.pkl',
 'CNA_10xmultiome_2.FULL': 'downstream_analysis/imputed_acc_objs/CNA_10xmultiome_2.FULL__imputed_acc_obs.pkl',
 'CNA_10xv11_1.FULL': 'downstream_analysis/imputed_acc_objs/CNA_10xv11_1.FULL__imputed_acc_obs.pkl',
 'CNA_10xv11_2.FULL': 'downstream_analysis/imputed_acc_objs/CNA_10xv11_

<IPython.core.display.Javascript object>

In [11]:
hvr_path_dict = {x.split('/')[-1].split(f'__')[0]:x for x in sorted(glob.glob(f'downstream_analysis/HVRs/*bed'))}
hvr_path_dict

{'BIO_ddseq_1.FULL': 'downstream_analysis/HVRs/BIO_ddseq_1.FULL__HVRs.bed',
 'BIO_ddseq_2.FULL': 'downstream_analysis/HVRs/BIO_ddseq_2.FULL__HVRs.bed',
 'BIO_ddseq_3.FULL': 'downstream_analysis/HVRs/BIO_ddseq_3.FULL__HVRs.bed',
 'BIO_ddseq_4.FULL': 'downstream_analysis/HVRs/BIO_ddseq_4.FULL__HVRs.bed',
 'BRO_mtscatac_1.FULL': 'downstream_analysis/HVRs/BRO_mtscatac_1.FULL__HVRs.bed',
 'BRO_mtscatac_2.FULL': 'downstream_analysis/HVRs/BRO_mtscatac_2.FULL__HVRs.bed',
 'CNA_10xmultiome_1.FULL': 'downstream_analysis/HVRs/CNA_10xmultiome_1.FULL__HVRs.bed',
 'CNA_10xmultiome_2.FULL': 'downstream_analysis/HVRs/CNA_10xmultiome_2.FULL__HVRs.bed',
 'CNA_10xv11_1.FULL': 'downstream_analysis/HVRs/CNA_10xv11_1.FULL__HVRs.bed',
 'CNA_10xv11_2.FULL': 'downstream_analysis/HVRs/CNA_10xv11_2.FULL__HVRs.bed',
 'CNA_10xv11_3.FULL': 'downstream_analysis/HVRs/CNA_10xv11_3.FULL__HVRs.bed',
 'CNA_10xv11_4.FULL': 'downstream_analysis/HVRs/CNA_10xv11_4.FULL__HVRs.bed',
 'CNA_10xv11_5.FULL': 'downstream_analysis/H

<IPython.core.display.Javascript object>

In [12]:
for sample in cistopic_obj_path_dict.keys():
# for sample  in ['SAN_10xmultiome_1.FIXEDCELLS']:
    if not os.path.exists(f'male_female_DARs/{sample}'):
        cto_path = cistopic_obj_path_dict[sample]
        if os.path.isfile(cto_path):
            print(f"Loading {cto_path}")
            with open(cto_path, 'rb') as f:
                cto = pickle.load(f)
                
            cto.cell_names = cto.cell_data.index
            if [x.split('-')[0] for x in cto.selected_model.cell_topic.columns] == [x.split('___')[0] for x in list(cto.cell_names)]:
                cto.selected_model.cell_topic.columns =  list(cto.cell_names)
                
        if 'fmx_sample' in cto.cell_data.columns:
            acc_path = normalized_imputed_acc_obj_path_dict[sample]

            if os.path.isfile(acc_path):
                print(f"Loading {acc_path}")
                with open(acc_path, 'rb') as f:
                    normalized_imputed_acc_obj = pickle.load(f)


            hvr_pkl_path = f'downstream_analysis/HVRs/{sample}__HVRs.pkl'

            if os.path.exists(hvr_pkl_path):
                print(f"Loading {hvr_pkl_path}")
                with open(hvr_pkl_path, 'rb') as f:
                    variable_regions = pickle.load(f)
            else:
                print("HVRs do not exist!")

            cto.cell_data['fmx_sample_consensus_cell_type'] = cto.cell_data['fmx_sample'] + "_" + cto.cell_data["consensus_cell_type"]

            contrasts = []
            for cell_type in cto.cell_data['consensus_cell_type'].unique():
                print(cell_type)
                ct_a = 'sampleA_' + cell_type
                ct_b = 'sampleB_' + cell_type
                contrast = [[[ct_a], [ct_b]], [[ct_b], [ct_a]]]
                contrasts = contrasts + contrast

            markers_dict = find_diff_features(
                cto,
                normalized_imputed_acc_obj,
                variable='fmx_sample_consensus_cell_type',
                var_features=variable_regions,
                contrasts=contrasts,
                adjpval_thr=0.05,
                log2fc_thr=np.log2(1.2),
                n_cpu=10,
            )
            
            if not os.path.exists(f'male_female_DARs/{sample}'):
                os.mkdir(f'male_female_DARs/{sample}')
                
            with open(
                f"male_female_DARs/{sample}/{sample}__DARs_dict_1-2xfoldchange.pkl",'wb'
            ) as f:
                pickle.dump(markers_dict, f, protocol=4)
                        
            for contrast in markers_dict.keys():
                markers = markers_dict[contrast].index.tolist()
                df = pd.DataFrame(markers)
                if len(df) == 0:
                    print(f"no DARs found for {contrast} in {sample}")
                else:
                    df[[0,1]] = df[0].str.split(':',expand=True)
                    df[[1,2]] = df[1].str.split('-',expand=True)
                    df[3] = markers_dict[contrast].index.tolist()
                    df[3] = contrast.replace(' ', '_')# + '_' + df[3].astype(str)
                    score = markers_dict[contrast]['Log2FC']
                    score = score.reset_index(drop=True)
                    df[4] = score
                    df[5] = '.'
                    pval = markers_dict[contrast]['Adjusted_pval']
                    pval = pval.reset_index(drop=True)
                    df[6] = pval
                    name = contrast.replace(' ', '_')

                    df.to_csv(f'male_female_DARs/{sample}/{sample}__{contrast.replace(" ", "_")}__1-2xfoldchange_DARs.bed', sep='\t', header=False, index=False)

Loading cistopic_objects/BIO_ddseq_1.FULL__cto.scrublet0-4.fmx.singlets.model_20topics.dimreduc.consensus.pkl
Loading cistopic_objects/BIO_ddseq_2.FULL__cto.scrublet0-4.fmx.singlets.model_20topics.dimreduc.consensus.pkl
Loading cistopic_objects/BIO_ddseq_3.FULL__cto.scrublet0-4.fmx.singlets.model_21topics.dimreduc.consensus.pkl
Loading cistopic_objects/BIO_ddseq_4.FULL__cto.scrublet0-4.fmx.singlets.model_19topics.dimreduc.consensus.pkl
Loading cistopic_objects/BRO_mtscatac_1.FULL__cto.scrublet0-4.fmx.singlets.model_18topics.dimreduc.consensus.pkl
Loading downstream_analysis/imputed_acc_objs/BRO_mtscatac_1.FULL__imputed_acc_obs.pkl
Loading downstream_analysis/HVRs/BRO_mtscatac_1.FULL__HVRs.pkl
Dendritic cell
CD14+ monocyte
B cell
CD4+ T cell
Cytotoxic T cell
Natural killer cell
CD16+ monocyte


2022-09-30 14:17:14,581	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265


(markers_ray pid=108142) 2022-09-30 14:17:44,203 cisTopic     INFO     Formatting data for sampleA_Dendritic cell_VS_sampleB_Dendritic cell
(markers_ray pid=108142) 2022-09-30 14:17:44,756 cisTopic     INFO     Computing p-value for sampleA_Dendritic cell_VS_sampleB_Dendritic cell
(markers_ray pid=108143) 2022-09-30 14:17:44,742 cisTopic     INFO     Formatting data for sampleB_Dendritic cell_VS_sampleA_Dendritic cell
(markers_ray pid=108143) 2022-09-30 14:17:45,314 cisTopic     INFO     Computing p-value for sampleB_Dendritic cell_VS_sampleA_Dendritic cell
(markers_ray pid=108141) 2022-09-30 14:17:45,574 cisTopic     INFO     Formatting data for sampleA_CD14+ monocyte_VS_sampleB_CD14+ monocyte
(markers_ray pid=108140) 2022-09-30 14:17:46,231 cisTopic     INFO     Formatting data for sampleB_CD14+ monocyte_VS_sampleA_CD14+ monocyte
(markers_ray pid=108139) 2022-09-30 14:17:46,843 cisTopic     INFO     Formatting data for sampleA_B cell_VS_sampleB_B cell
(markers_ray pid=108137) 2022-09

2022-09-30 14:27:04,761	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265


(markers_ray pid=109084) 2022-09-30 14:27:34,497 cisTopic     INFO     Formatting data for sampleA_CD14+ monocyte_VS_sampleB_CD14+ monocyte
(markers_ray pid=109083) 2022-09-30 14:27:35,048 cisTopic     INFO     Formatting data for sampleB_CD14+ monocyte_VS_sampleA_CD14+ monocyte
(markers_ray pid=109082) 2022-09-30 14:27:35,752 cisTopic     INFO     Formatting data for sampleA_Natural killer cell_VS_sampleB_Natural killer cell
(markers_ray pid=109081) 2022-09-30 14:27:36,354 cisTopic     INFO     Formatting data for sampleB_Natural killer cell_VS_sampleA_Natural killer cell
(markers_ray pid=109078) 2022-09-30 14:27:37,015 cisTopic     INFO     Formatting data for sampleA_CD4+ T cell_VS_sampleB_CD4+ T cell
(markers_ray pid=109082) 2022-09-30 14:27:37,480 cisTopic     INFO     Computing p-value for sampleA_Natural killer cell_VS_sampleB_Natural killer cell
(markers_ray pid=109079) 2022-09-30 14:27:37,662 cisTopic     INFO     Formatting data for sampleB_CD4+ T cell_VS_sampleA_CD4+ T cell


...

2022-09-30 18:16:21,180	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265


(markers_ray pid=135690) 2022-09-30 18:16:51,265 cisTopic     INFO     Formatting data for sampleB_CD4+ T cell_VS_sampleA_CD4+ T cell
(markers_ray pid=135691) 2022-09-30 18:16:51,311 cisTopic     INFO     Formatting data for sampleA_CD4+ T cell_VS_sampleB_CD4+ T cell
(markers_ray pid=135689) 2022-09-30 18:16:51,533 cisTopic     INFO     Formatting data for sampleA_CD14+ monocyte_VS_sampleB_CD14+ monocyte
(markers_ray pid=135688) 2022-09-30 18:16:51,954 cisTopic     INFO     Formatting data for sampleB_CD14+ monocyte_VS_sampleA_CD14+ monocyte
(markers_ray pid=135687) 2022-09-30 18:16:52,562 cisTopic     INFO     Formatting data for sampleA_CD16+ monocyte_VS_sampleB_CD16+ monocyte
(markers_ray pid=135686) 2022-09-30 18:16:53,182 cisTopic     INFO     Formatting data for sampleB_CD16+ monocyte_VS_sampleA_CD16+ monocyte
(markers_ray pid=135687) 2022-09-30 18:16:53,544 cisTopic     INFO     Computing p-value for sampleA_CD16+ monocyte_VS_sampleB_CD16+ monocyte
(markers_ray pid=135685) 2022-

2022-09-30 18:24:59,920	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265


(markers_ray pid=136574) 2022-09-30 18:25:28,941 cisTopic     INFO     Formatting data for sampleA_Dendritic cell_VS_sampleB_Dendritic cell
(markers_ray pid=136574) 2022-09-30 18:25:29,528 cisTopic     INFO     Computing p-value for sampleA_Dendritic cell_VS_sampleB_Dendritic cell
(markers_ray pid=136573) 2022-09-30 18:25:29,454 cisTopic     INFO     Formatting data for sampleB_Dendritic cell_VS_sampleA_Dendritic cell
(markers_ray pid=136573) 2022-09-30 18:25:30,006 cisTopic     INFO     Computing p-value for sampleB_Dendritic cell_VS_sampleA_Dendritic cell
(markers_ray pid=136572) 2022-09-30 18:25:30,053 cisTopic     INFO     Formatting data for sampleA_CD4+ T cell_VS_sampleB_CD4+ T cell
(markers_ray pid=136570) 2022-09-30 18:25:30,519 cisTopic     INFO     Formatting data for sampleB_CD4+ T cell_VS_sampleA_CD4+ T cell
(markers_ray pid=136571) 2022-09-30 18:25:31,081 cisTopic     INFO     Formatting data for sampleA_CD14+ monocyte_VS_sampleB_CD14+ monocyte
(markers_ray pid=136568) 202

<IPython.core.display.Javascript object>

# Calculate DARs between male and female, now for each cell type

In [13]:
normalized_imputed_acc_obj_path_dict

{'BIO_ddseq_1.FULL': 'downstream_analysis/imputed_acc_objs/BIO_ddseq_1.FULL__imputed_acc_obs.pkl',
 'BIO_ddseq_2.FULL': 'downstream_analysis/imputed_acc_objs/BIO_ddseq_2.FULL__imputed_acc_obs.pkl',
 'BIO_ddseq_3.FULL': 'downstream_analysis/imputed_acc_objs/BIO_ddseq_3.FULL__imputed_acc_obs.pkl',
 'BIO_ddseq_4.FULL': 'downstream_analysis/imputed_acc_objs/BIO_ddseq_4.FULL__imputed_acc_obs.pkl',
 'BRO_mtscatac_1.FULL': 'downstream_analysis/imputed_acc_objs/BRO_mtscatac_1.FULL__imputed_acc_obs.pkl',
 'BRO_mtscatac_2.FULL': 'downstream_analysis/imputed_acc_objs/BRO_mtscatac_2.FULL__imputed_acc_obs.pkl',
 'CNA_10xmultiome_1.FULL': 'downstream_analysis/imputed_acc_objs/CNA_10xmultiome_1.FULL__imputed_acc_obs.pkl',
 'CNA_10xmultiome_2.FULL': 'downstream_analysis/imputed_acc_objs/CNA_10xmultiome_2.FULL__imputed_acc_obs.pkl',
 'CNA_10xv11_1.FULL': 'downstream_analysis/imputed_acc_objs/CNA_10xv11_1.FULL__imputed_acc_obs.pkl',
 'CNA_10xv11_2.FULL': 'downstream_analysis/imputed_acc_objs/CNA_10xv11_

<IPython.core.display.Javascript object>

In [14]:
normalized_imputed_acc_obj_path_dict = {x.split('/')[-1].split(f'__')[0]:x for x in sorted(glob.glob(f'downstream_analysis/imputed_acc_objs/*__imputed_acc_obs.pkl'))}
normalized_imputed_acc_obj_path_dict

{'BIO_ddseq_1.FULL': 'downstream_analysis/imputed_acc_objs/BIO_ddseq_1.FULL__imputed_acc_obs.pkl',
 'BIO_ddseq_2.FULL': 'downstream_analysis/imputed_acc_objs/BIO_ddseq_2.FULL__imputed_acc_obs.pkl',
 'BIO_ddseq_3.FULL': 'downstream_analysis/imputed_acc_objs/BIO_ddseq_3.FULL__imputed_acc_obs.pkl',
 'BIO_ddseq_4.FULL': 'downstream_analysis/imputed_acc_objs/BIO_ddseq_4.FULL__imputed_acc_obs.pkl',
 'BRO_mtscatac_1.FULL': 'downstream_analysis/imputed_acc_objs/BRO_mtscatac_1.FULL__imputed_acc_obs.pkl',
 'BRO_mtscatac_2.FULL': 'downstream_analysis/imputed_acc_objs/BRO_mtscatac_2.FULL__imputed_acc_obs.pkl',
 'CNA_10xmultiome_1.FULL': 'downstream_analysis/imputed_acc_objs/CNA_10xmultiome_1.FULL__imputed_acc_obs.pkl',
 'CNA_10xmultiome_2.FULL': 'downstream_analysis/imputed_acc_objs/CNA_10xmultiome_2.FULL__imputed_acc_obs.pkl',
 'CNA_10xv11_1.FULL': 'downstream_analysis/imputed_acc_objs/CNA_10xv11_1.FULL__imputed_acc_obs.pkl',
 'CNA_10xv11_2.FULL': 'downstream_analysis/imputed_acc_objs/CNA_10xv11_

<IPython.core.display.Javascript object>

In [15]:
hvr_path_dict = {x.split('/')[-1].split(f'__')[0]:x for x in sorted(glob.glob(f'downstream_analysis/HVRs/*bed'))}
hvr_path_dict

{'BIO_ddseq_1.FULL': 'downstream_analysis/HVRs/BIO_ddseq_1.FULL__HVRs.bed',
 'BIO_ddseq_2.FULL': 'downstream_analysis/HVRs/BIO_ddseq_2.FULL__HVRs.bed',
 'BIO_ddseq_3.FULL': 'downstream_analysis/HVRs/BIO_ddseq_3.FULL__HVRs.bed',
 'BIO_ddseq_4.FULL': 'downstream_analysis/HVRs/BIO_ddseq_4.FULL__HVRs.bed',
 'BRO_mtscatac_1.FULL': 'downstream_analysis/HVRs/BRO_mtscatac_1.FULL__HVRs.bed',
 'BRO_mtscatac_2.FULL': 'downstream_analysis/HVRs/BRO_mtscatac_2.FULL__HVRs.bed',
 'CNA_10xmultiome_1.FULL': 'downstream_analysis/HVRs/CNA_10xmultiome_1.FULL__HVRs.bed',
 'CNA_10xmultiome_2.FULL': 'downstream_analysis/HVRs/CNA_10xmultiome_2.FULL__HVRs.bed',
 'CNA_10xv11_1.FULL': 'downstream_analysis/HVRs/CNA_10xv11_1.FULL__HVRs.bed',
 'CNA_10xv11_2.FULL': 'downstream_analysis/HVRs/CNA_10xv11_2.FULL__HVRs.bed',
 'CNA_10xv11_3.FULL': 'downstream_analysis/HVRs/CNA_10xv11_3.FULL__HVRs.bed',
 'CNA_10xv11_4.FULL': 'downstream_analysis/HVRs/CNA_10xv11_4.FULL__HVRs.bed',
 'CNA_10xv11_5.FULL': 'downstream_analysis/H

<IPython.core.display.Javascript object>

In [16]:
for sample in cistopic_obj_path_dict.keys():
# for sample  in ['SAN_10xmultiome_1.FIXEDCELLS']:
    if not os.path.exists(f'male_female_DARs/{sample}'):
        cto_path = cistopic_obj_path_dict[sample]
        if os.path.isfile(cto_path):
            print(f"Loading {cto_path}")
            with open(cto_path, 'rb') as f:
                cto = pickle.load(f)
                
            cto.cell_names = cto.cell_data.index
            if [x.split('-')[0] for x in cto.selected_model.cell_topic.columns] == [x.split('___')[0] for x in list(cto.cell_names)]:
                cto.selected_model.cell_topic.columns =  list(cto.cell_names)
                
        if 'fmx_sample' in cto.cell_data.columns:
            acc_path = normalized_imputed_acc_obj_path_dict[sample]

            if os.path.isfile(acc_path):
                print(f"Loading {acc_path}")
                with open(acc_path, 'rb') as f:
                    normalized_imputed_acc_obj = pickle.load(f)


            hvr_pkl_path = f'downstream_analysis/HVRs/{sample}__HVRs.pkl'

            if os.path.exists(hvr_pkl_path):
                print(f"Loading {hvr_pkl_path}")
                with open(hvr_pkl_path, 'rb') as f:
                    variable_regions = pickle.load(f)
            else:
                print("HVRs do not exist!")

            cto.cell_data['fmx_sample_consensus_cell_type'] = cto.cell_data['fmx_sample'] + "_" + cto.cell_data["consensus_cell_type"]

            contrasts = []
            for cell_type in cto.cell_data['consensus_cell_type'].unique():
                print(cell_type)
                ct_a = 'sampleA_' + cell_type
                ct_b = 'sampleB_' + cell_type
                contrast = [[[ct_a], [ct_b]], [[ct_b], [ct_a]]]
                contrasts = contrasts + contrast

            markers_dict = find_diff_features(
                cto,
                normalized_imputed_acc_obj,
                variable='fmx_sample_consensus_cell_type',
                var_features=variable_regions,
                contrasts=contrasts,
                adjpval_thr=0.05,
                log2fc_thr=np.log2(1.2),
                n_cpu=10,
            )
            
            if not os.path.exists(f'male_female_DARs/{sample}'):
                os.mkdir(f'male_female_DARs/{sample}')
                
            with open(
                f"male_female_DARs/{sample}/{sample}__DARs_dict_1-2xfoldchange.pkl",'wb'
            ) as f:
                pickle.dump(markers_dict, f, protocol=4)
                        
            for contrast in markers_dict.keys():
                markers = markers_dict[contrast].index.tolist()
                df = pd.DataFrame(markers)
                if len(df) == 0:
                    print(f"no DARs found for {contrast} in {sample}")
                else:
                    df[[0,1]] = df[0].str.split(':',expand=True)
                    df[[1,2]] = df[1].str.split('-',expand=True)
                    df[3] = markers_dict[contrast].index.tolist()
                    df[3] = contrast.replace(' ', '_')# + '_' + df[3].astype(str)
                    score = markers_dict[contrast]['Log2FC']
                    score = score.reset_index(drop=True)
                    df[4] = score
                    df[5] = '.'
                    pval = markers_dict[contrast]['Adjusted_pval']
                    pval = pval.reset_index(drop=True)
                    df[6] = pval
                    name = contrast.replace(' ', '_')

                    df.to_csv(f'male_female_DARs/{sample}/{sample}__{contrast.replace(" ", "_")}__1-2xfoldchange_DARs.bed', sep='\t', header=False, index=False)

Loading cistopic_objects/BIO_ddseq_1.FULL__cto.scrublet0-4.fmx.singlets.model_20topics.dimreduc.consensus.pkl
Loading cistopic_objects/BIO_ddseq_2.FULL__cto.scrublet0-4.fmx.singlets.model_20topics.dimreduc.consensus.pkl
Loading cistopic_objects/BIO_ddseq_3.FULL__cto.scrublet0-4.fmx.singlets.model_21topics.dimreduc.consensus.pkl
Loading cistopic_objects/BIO_ddseq_4.FULL__cto.scrublet0-4.fmx.singlets.model_19topics.dimreduc.consensus.pkl
Loading cistopic_objects/OHS_s3atac_1.FULL__cto.scrublet0-4.fmx.singlets.model_14topics.dimreduc.consensus.pkl
Loading cistopic_objects/OHS_s3atac_2.FULL__cto.scrublet0-4.fmx.singlets.model_19topics.dimreduc.consensus.pkl
Loading cistopic_objects/TXG_10xv11_1.FULL__cto.scrublet0-4.fmx.singlets.model_13topics.dimreduc.consensus.pkl
Loading cistopic_objects/TXG_10xv2_1.FULL__cto.scrublet0-4.fmx.singlets.model_21topics.dimreduc.consensus.pkl
Loading cistopic_objects/TXG_10xv2_2.FULL__cto.scrublet0-4.fmx.singlets.model_22topics.dimreduc.consensus.pkl
Loading

<IPython.core.display.Javascript object>